<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [0]:
import pandas as pd

In [4]:
from google.colab import files

uploaded = files.upload()

Saving review_sample.json to review_sample (1).json


In [0]:
yelp = pd.read_json(uploaded['review_sample.json'], lines=True)

In [7]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [0]:
!python -m spacy download en_core_web_md

In [0]:
import spacy
import en_core_web_md
nlp = en_core_web_md.load()

In [0]:
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [0]:
def tokenize(doc):
    
    return [token.text for token in tokenizer(doc)]

In [0]:
tokenize(yelp.text[0])

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english", max_features=5000)

dtm = tfidf.fit_transform(yelp.text)

docs = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
docs.head()

,00,000,00pm,07,10,100,1000,101,10pm,11,110,11am,11pm,12,120,13,14,15,150,16,17,18,19,1pm,1st,20,200,2006,2011,2012,2013,2014,2015,2016,2017,2018,21,22,23,24,...,wrong,wrote,wtf,www,wynn,ya,yard,yay,yeah,year,years,yell,yelled,yelling,yellow,yellowtail,yelp,yelpers,yep,yes,yesterday,yikes,yo,yoga,yogurt,yolk,yonge,york,young,younger,yuck,yuk,yum,yummy,yup,zero,zone,zoo,zucchini,était
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086242,...,0.0,0.0,0.0,0.0,0.10146,0.0,0.0,0.0,0.0,0.066314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145412,0.0,0.0,0.0,0.130277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [20]:
fake_review = ['This place sucks!']
new = tfidf.transform(fake_review)
nn.kneighbors(new.todense())

(array([[0.9502405 , 1.        , 1.        , 1.00785388, 1.02244851,
         1.08412796, 1.1294588 , 1.16791855, 1.17189012, 1.20690446]]),
 array([[5423, 6311, 6204, 3230, 9035, 3411, 6167, 5796, 5105, 4910]]))

In [25]:
yelp.text[3230]

'This place just sucks. Food was terrible. Service was even worse some how. Just skip it all together.'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [0]:
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier

In [0]:
# Create pipeline
vect = TfidfVectorizer(stop_words='english')
sgdc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

In [29]:
# Fit pipeline
pipe.fit(yelp.text, yelp.stars)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
           

In [31]:
# Prediction -> (1 star!)
pipe.predict(fake_review)

array([1])

In [0]:
# Pipe tuning
from sklearn.model_selection import GridSearchCV

In [0]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1),
    'clf__max_iter': (20, 10, 100)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)

In [35]:
grid_search.fit(yelp.text, yelp.stars)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   41.4s finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [0]:
import numpy as np
import gensim
import os
import re

from gensim.utils import smart_open, simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [0]:
def tokenize2(text):
  return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [0]:
id2word = corpora.Dictionary([tokenize2(row) for row in yelp.text])

In [0]:
id2word.filter_extremes(no_below=10, no_above=0.75)

In [0]:
corpus = [id2word.doc2bow(tokenize2(text)) for text in yelp.text]

In [0]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [0]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [0]:
topics = [' '.join(t[0:5]) for t in words]

In [59]:
print(topics[0])

food good place great service


In [61]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [63]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.148810  0.042519       1        1  13.249545
7      0.025523  0.097172       2        1  11.806889
8     -0.126384  0.080713       3        1   8.827474
4     -0.145748  0.025841       4        1   8.376736
0      0.081217 -0.051936       5        1   8.259304
14    -0.182866  0.016442       6        1   8.085751
12     0.141552  0.014692       7        1   7.563570
13    -0.121733 -0.070702       8        1   5.022072
10     0.009358 -0.150859       9        1   4.880160
6      0.045352  0.007773      10        1   4.701841
3     -0.131607 -0.057363      11        1   4.648199
2      0.119546 -0.050774      12        1   4.422679
9      0.029238  0.047758      13        1   3.893789
5      0.116663  0.002518      14        1   3.269889
1     -0.008923  0.046207      15        1   2.992108, topic_info=     Category         Freq        Term        Total  loglift  logprob
133   Default  4934.000000        food  4934.000000  30.0000  30.0000
188   Default  4239.000000       great  4239.000000  29.0000  29.0000
94    Default  1013.000000        room  1013.000000  28.0000  28.0000
1667  Default   601.000000         car   601.000000  27.0000  27.0000
122   Default  4730.000000       place  4730.000000  26.0000  26.0000
682   Default   524.000000       sushi   524.000000  25.0000  25.0000
667   Default   609.000000       cream   609.000000  24.0000  24.0000
189   Default   325.000000        hair   325.000000  23.0000  23.0000
37    Default  4597.000000        good  4597.000000  22.0000  22.0000
1039  Default  1447.000000     chicken  1447.000000  21.0000  21.0000
904   Default   486.000000        beer   486.000000  20.0000  20.0000
671   Default   504.000000         ice   504.000000  19.0000  19.0000
992   Default   609.000000      coffee   609.000000  18.0000  18.0000
1509  Default   464.000000       hotel   464.000000  17.0000  17.0000
42    Default  3639.000000     service  3639.000000  16.0000  16.0000
1029  Default  1577.000000     ordered  1577.000000  15.0000  15.0000
348   Default   796.000000       pizza   796.000000  14.0000  14.0000
309   Default   864.000000      cheese   864.000000  13.0000  13.0000
318   Default  1251.000000   delicious  1251.000000  12.0000  12.0000
126   Default  1045.000000         bar  1045.000000  11.0000  11.0000
176   Default  1353.000000     amazing  1353.000000  10.0000  10.0000
43    Default  1552.000000       staff  1552.000000   9.0000   9.0000
570   Default  1376.000000  restaurant  1376.000000   8.0000   8.0000
111   Default  1849.000000        best  1849.000000   7.0000   7.0000
392   Default   809.000000    customer   809.000000   6.0000   6.0000
638   Default  1640.000000        love  1640.000000   5.0000   5.0000
353   Default   858.000000       sauce   858.000000   4.0000   4.0000
104   Default  1931.000000          ve  1931.000000   3.0000   3.0000
168   Default  1163.000000   recommend  1163.000000   2.0000   2.0000
1257  Default   274.000000       rolls   274.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
262   Topic15    56.348984       salon   229.900406   2.1031  -5.4225
126   Topic15   113.521820         bar  1045.549805   1.2889  -4.7221
2711  Topic15    19.369715        cuts    49.607353   2.5688  -6.4904
1777  Topic15    21.124966   dumplings    62.727070   2.4209  -6.4036
104   Topic15   110.810280          ve  1931.165405   0.6511  -4.7463
120   Topic15    86.950348      people  1435.017456   0.7056  -4.9887
176   Topic15    82.483719     amazing  1353.539429   0.7113  -5.0415
37    Topic15   136.698792        good  4597.698730  -0.0063  -4.5363
302   Topic15    57.865456     awesome   665.200867   1.0672  -5.3960
106   Topic15    81.094337        went  1494.936035   0.5950  -5.0585
208   Topic15    69.996513         way  1098.210449   0.7562  -5.2056
786   Topic15    57.028343   

In [0]:
distro = [lda[d] for d in corpus]

In [66]:
distro[0]

[(1, 0.15415904), (3, 0.35893506), (8, 0.46739364)]

In [0]:
def update(doc):
  d_dist = {k:0 for k in range(0,15)}
  for t in doc:
    d_dist[t[0]] = t[1]
  return d_dist

new_distro = [update(d) for d in distro]

In [0]:
df = pd.DataFrame.from_records(new_distro)
df.columns = topics

In [76]:
df.head()

,food good place great service,hair good bar cut ve,place great sushi food beer,car service great pizza customer,service time place customer like,food great service good time,coffee like food good great,food ordered good like order,room hotel got time night,good like place time great,great place food best friendly,chicken good food place like,good cream ice cheese pizza,place great staff time recommend,time like said new ve
0,0.000000,0.154159,0.000000,0.358935,0.0,0.0,0.000000,0.0,0.467394,0.0,0.00000,0.000000,0.0,0.000000,0.0
1,0.383009,0.000000,0.000000,0.000000,0.0,0.0,0.265058,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.290395,0.0
2,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.843066,0.0,0.00000,0.000000,0.0,0.146101,0.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.765106,0.0,0.18928,0.000000,0.0,0.000000,0.0
4,0.534650,0.000000,0.317138,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.128212,0.0,0.000000,0.0


In [0]:
df['biz_id'] = yelp['business_id']

In [78]:
df.head()

,food good place great service,hair good bar cut ve,place great sushi food beer,car service great pizza customer,service time place customer like,food great service good time,coffee like food good great,food ordered good like order,room hotel got time night,good like place time great,great place food best friendly,chicken good food place like,good cream ice cheese pizza,place great staff time recommend,time like said new ve,biz_id
0,0.000000,0.154159,0.000000,0.358935,0.0,0.0,0.000000,0.0,0.467394,0.0,0.00000,0.000000,0.0,0.000000,0.0,nDuEqIyRc8YKS1q1fX0CZg
1,0.383009,0.000000,0.000000,0.000000,0.0,0.0,0.265058,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.290395,0.0,eMYeEapscbKNqUDCx705hg
2,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.843066,0.0,0.00000,0.000000,0.0,0.146101,0.0,6Q7-wkCPc1KF75jZLOTcMw
3,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.765106,0.0,0.18928,0.000000,0.0,0.000000,0.0,k3zrItO4l9hwfLRwHBDc9w
4,0.534650,0.000000,0.317138,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.128212,0.0,0.000000,0.0,6hpfRwGlOzbNv7k5eP9rsQ


In [86]:
df2 = df.groupby('biz_id').mean().sort_values(by='food good place great service', ascending=False)
df2[df2['food good place great service'] > 0.9]

,food good place great service,hair good bar cut ve,place great sushi food beer,car service great pizza customer,service time place customer like,food great service good time,coffee like food good great,food ordered good like order,room hotel got time night,good like place time great,great place food best friendly,chicken good food place like,good cream ice cheese pizza,place great staff time recommend,time like said new ve
biz_id,,,,,,,,,,,,,,,
r3kFRJuy7CTxamZycndqGw,0.989964,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
FI-8LPGkc8D_hwx0l0zyyg,0.988477,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
iskeoO9kfBh1HoYHXuNFwA,0.985859,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
S3-JpB5dBhMXmauMqeoN-g,0.985641,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
XIgTZdZqJLuaYnaWG3jyFw,0.985641,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
hY2vkn2TFxegPyqTdlKOcA,0.984946,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
HNAH23RNUTTivzhebQTozw,0.982051,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
rfgln6-7WFCu7S7fOoqMdA,0.979710,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
TL3hKj7fqMdFqSwBSTKX-w,0.979259,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


**Interpretation**

In the amount of time I had left on this sprint I was able to notice that many of the topics included positive words like "good", "great", etc. It seems like the restaurants that were sampled tended to have positive reviews where the customers were really pleased with food quality and service. I checked one topic ("food good place great service" ) in the previous cell where I wanted to know how many restaurants had an above 90 distribution in that particular topic. It turns out that 134 satisfied that requirement. So just based off these initial findings I would say that the reviews tend to skew positively in the areas they polled. Of course I haven't checked how many restaurants had low density distributions for the seemingly positive topics but if I had time that is something I would dive into to further analyze the results. I would also work to enhance the pre-processing and visualizations.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)